<a href="https://colab.research.google.com/github/kid-116/AI-in-WebDev-SMP/blob/main/MatrixFactorization/RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Recommender Sysytem</h1>
<h2>by kid116</h2>

<h3>Importing Libraries</h3>

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import random

<h3>The Dataset</h3>

<h4>Movies</h4>

In [3]:
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'name': ['The Godfather', 'Shawshank Redemption', 'Soul', 'Avengers', 'Inception', 'Titanic', "Schindler's List", 'American Made', 'Fight Club', 'Kingsglaive'],
    'action': [5, 1, 1, 10, 8, 1, 0, 9, 9, 9],
    'comedy': [0, 1, 6, 8, 0, 3, 0, 8, 0, 0],
    'crime': [10, 8, 1, 1, 7, 3, 6, 9, 9, 3],
    'romance': [1, 1, 3, 1, 3, 10, 0, 6, 3, 0],
    'thriller': [7, 6, 2, 2, 7, 1, 0, 8, 9, 9],
    'fantasy': [0, 0, 7, 8, 8, 0, 0, 0, 0, 10],
    'drama': [8, 1, 0, 0, 0, 8, 9, 1, 0, 0],
}

df = pd.DataFrame(data)

df.head()

,id,name,action,comedy,crime,romance,thriller,fantasy,drama
0,1,The Godfather,5,0,10,1,7,0,8
1,2,Shawshank Redemption,1,1,8,1,6,0,1
2,3,Soul,1,6,1,3,2,7,0
3,4,Avengers,10,8,1,1,2,8,0
4,5,Inception,8,0,7,3,7,8,0


<h4>Ratings</h4>

In [4]:
ratings=np.array([[
    [0, 5, 0, 4, 5, 4, 0, 0, 0, 0], # Madhav
    [4, 0, 0, 5, 0, 4, 0, 4, 0, 0], # Rounak Somani
    [4, 4, 3, 3, 4, 0, 0, 0, 4, 0], # Tejas
    [1, 5, 3, 2, 5, 4, 5, 5, 5, 5], # Mehul
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # Satvik
    [5, 5, 5, 4, 5, 4, 5, 0, 5, 0], # Sapta
    # [4.9, 4.2, 0, 1, 4.5, 4, 0, 2.5, 4.7, 0], # Janmansh
    [4, 0, 0, 4, 5, 4, 0, 0, 3, 0], # Samyak
    [0, 0, 0, 4, 3, 4, 0, 0, 0, 0], # Sashank
    [0, 5, 0, 3, 5, 0, 0, 0, 3, 0], # Pratham
    [0, 5, 0, 4, 5, 0, 0, 0, 0, 0], # Samhita
    # [0, 4, 0, 4, 5, 4, 0, 0, 4, 0], # Rakshit
]])

print(ratings)
print(ratings.shape)

[[[0 5 0 4 5 4 0 0 0 0]
  [4 0 0 5 0 4 0 4 0 0]
  [4 4 3 3 4 0 0 0 4 0]
  [1 5 3 2 5 4 5 5 5 5]
  [0 0 0 0 0 0 0 0 0 0]
  [5 5 5 4 5 4 5 0 5 0]
  [4 0 0 4 5 4 0 0 3 0]
  [0 0 0 4 3 4 0 0 0 0]
  [0 5 0 3 5 0 0 0 3 0]
  [0 5 0 4 5 0 0 0 0 0]]]
(1, 10, 10)


<h3>Extracting Features</h3>

In [5]:
# Some Constants
user_count = ratings.shape[1]
movie_count = df.shape[0]
k = df.shape[1] - 2 # Number of genres

print('Users:', user_count)
print('Movies:', movie_count)

Users: 10
Movies: 10


In [6]:
V = df.to_numpy()[:, 2:]
V /= 10 # Normalization
print(V)

[[0.5 0.0 1.0 0.1 0.7 0.0 0.8]
 [0.1 0.1 0.8 0.1 0.6 0.0 0.1]
 [0.1 0.6 0.1 0.3 0.2 0.7 0.0]
 [1.0 0.8 0.1 0.1 0.2 0.8 0.0]
 [0.8 0.0 0.7 0.3 0.7 0.8 0.0]
 [0.1 0.3 0.3 1.0 0.1 0.0 0.8]
 [0.0 0.0 0.6 0.0 0.0 0.0 0.9]
 [0.9 0.8 0.9 0.6 0.8 0.0 0.1]
 [0.9 0.0 0.9 0.3 0.9 0.0 0.0]
 [0.9 0.0 0.3 0.0 0.9 1.0 0.0]]


<h3>Taking Input</h3>

In [8]:
print("Please enter your ratings (0-5) for the following movies (0 if not seen):")
input_ratings = np.zeros(movie_count)
for i in range(movie_count):
  #  3
     print(df['name'][i], ":")
     input_ratings[i] = input() 

input_ratings = np.array([[
  input_ratings
]])

print(input_ratings)

Please enter your ratings (0-5) for the following movies (0 if not seen):
The Godfather :
0
Shawshank Redemption :
4
Soul :
0
Avengers :
4
Inception :
5
Titanic :
4
Schindler's List :
0
American Made :
0
Fight Club :
4
Kingsglaive :
0
[[[0. 4. 0. 4. 5. 4. 0. 0. 4. 0.]]]


In [9]:
ratings = np.hstack((ratings, input_ratings))
print(ratings)

[[[0. 5. 0. 4. 5. 4. 0. 0. 0. 0.]
  [4. 0. 0. 5. 0. 4. 0. 4. 0. 0.]
  [4. 4. 3. 3. 4. 0. 0. 0. 4. 0.]
  [1. 5. 3. 2. 5. 4. 5. 5. 5. 5.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [5. 5. 5. 4. 5. 4. 5. 0. 5. 0.]
  [4. 0. 0. 4. 5. 4. 0. 0. 3. 0.]
  [0. 0. 0. 4. 3. 4. 0. 0. 0. 0.]
  [0. 5. 0. 3. 5. 0. 0. 0. 3. 0.]
  [0. 5. 0. 4. 5. 0. 0. 0. 0. 0.]
  [0. 4. 0. 4. 5. 4. 0. 0. 4. 0.]]]


<h3>Training</h3>

In [10]:
movies = tf.Variable(V, dtype=tf.float64)
users  = tf.Variable(np.random.rand(user_count + 1,k), dtype=tf.float64)

In [12]:
mask = np.ma.masked_not_equal(ratings, 0).mask

ratings = tf.constant(ratings, dtype=tf.float64)

optimizer = tf.optimizers.Adam()

In [13]:
trainable_weights = [users, movies]

In [14]:
steps = 2000

for step in range(steps):
  with tf.GradientTape() as tape:
    predictions = tf.matmul(users, movies, transpose_b=True)
    predictions = tf.nn.relu(predictions)
    predictions = predictions*mask

    loss = tf.losses.mean_squared_error(ratings, predictions)
        
    gradients = tape.gradient(loss, trainable_weights)
    optimizer.apply_gradients(zip(gradients, trainable_weights))

predictions = tf.matmul(users, movies, transpose_b=True)
predictions = tf.nn.relu(predictions)

predictions = np.rint(predictions.numpy())

print(predictions)

[[4. 5. 5. 4. 5. 4. 5. 5. 4. 6.]
 [4. 5. 4. 5. 5. 4. 5. 4. 5. 6.]
 [4. 4. 3. 3. 4. 4. 5. 4. 4. 5.]
 [1. 4. 3. 2. 5. 4. 5. 5. 5. 5.]
 [3. 4. 3. 3. 4. 3. 3. 4. 3. 4.]
 [5. 5. 5. 4. 5. 4. 5. 5. 4. 6.]
 [4. 5. 4. 4. 5. 4. 5. 4. 4. 5.]
 [2. 3. 3. 4. 3. 4. 5. 3. 3. 4.]
 [4. 5. 4. 3. 5. 4. 5. 4. 4. 5.]
 [3. 5. 4. 4. 5. 5. 6. 5. 4. 6.]
 [3. 4. 4. 4. 5. 4. 5. 4. 4. 5.]]


<h3>Predictions</h3>

In [15]:
recs = {}

for i, pred in enumerate(predictions[-1]):
  if input_ratings[0][0][i] == 0:
    recs[df['name'][i]] = pred

recs = dict(sorted(recs.items(), key=lambda x:x[1]))
recs = list(recs.keys())[-5:]
list.reverse(recs)

print("Your recommendations are:")
for i, rec in enumerate(recs):
  print(f"\t{i + 1} {rec}")

Your recommendations are:
	1 Kingsglaive
	2 Schindler's List
	3 American Made
	4 Soul
	5 The Godfather
